In [1]:
"""What this does:
- Loads train.csv (labels + prompt for GroupKFold) and test.csv
- Loads per-model OOF train probs and test probs (any subset present)
  Expected columns in each predictions file: id, pA, pB, pTie  (probabilities)
- Rigorously validates inputs (alignment, ranges, sums, NaNs, class order)
- Builds a meta-dataset (concatenate per-model probs + engineered margins/entropy/lengths)
- Runs GroupKFold(by prompt) CV with two strategies:
    1) Constrained non-negative, sum-to-1 **logit-space** weighted average
    2) Multinomial Logistic Regression (stacking) on features
  Chooses the strategy with lower mean CV log-loss
- Refit on full train, predict test, and write:
    runs/<tag>/submission.csv  (id,winner_model_a,winner_model_b,winner_tie)
    runs/<tag>/oof_combined.csv
    runs/<tag>/cv_report.json
    runs/<tag>/blend_config.json
- Includes symmetry unit-tests (swap A/B) and tie calibration checks
- Gracefully degrades if some model files are missing"""

'What this does:\n- Loads train.csv (labels + prompt for GroupKFold) and test.csv\n- Loads per-model OOF train probs and test probs (any subset present)\n  Expected columns in each predictions file: id, pA, pB, pTie  (probabilities)\n- Rigorously validates inputs (alignment, ranges, sums, NaNs, class order)\n- Builds a meta-dataset (concatenate per-model probs + engineered margins/entropy/lengths)\n- Runs GroupKFold(by prompt) CV with two strategies:\n    1) Constrained non-negative, sum-to-1 **logit-space** weighted average\n    2) Multinomial Logistic Regression (stacking) on features\n  Chooses the strategy with lower mean CV log-loss\n- Refit on full train, predict test, and write:\n    runs/<tag>/submission.csv  (id,winner_model_a,winner_model_b,winner_tie)\n    runs/<tag>/oof_combined.csv\n    runs/<tag>/cv_report.json\n    runs/<tag>/blend_config.json\n- Includes symmetry unit-tests (swap A/B) and tie calibration checks\n- Gracefully degrades if some model files are missing'

In [2]:
import os
import sys
import json
import math
import time
import glob
import copy
import argparse
import logging
from dataclasses import dataclass, field
from typing import List, Dict, Optional, Tuple

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from scipy.special import logit, expit
from scipy.optimize import minimize

----------------------------

In [ ]:
CONFIG = {
    "train_csv": "./data/train.csv",
    "test_csv": "./data/test.csv",

    # === All models to blend ===
    "model_specs": [
        {
            "name": "teacher2_reward",   # Teacher 2: Reward model
            "train_path": "./teacher_logits_train_improved.csv",
            "test_path":  "./teacher_logits_test_improved.csv",
        },
        {
            "name": "deberta_head_only", # Teacher 1: Cross-encoder / DeBERTa baseline
            "train_path": "./deberta_oof_train.csv",
            "test_path":  "./submission_deberta_small_calibrated.csv",
        },
        {
            "name": "student_model",     # 🧠 Student: Distilled DeBERTa-v3-base
            "train_path": "./student_outputs/student_oof_probs.csv",
            "test_path":  "./student_outputs/student_test_probs.csv",
        },
    ],

    # === NEW output folder for the final ensemble ===
    "runs_dir": "./runs_final_ensemble",   # <--- 👈 This creates a separate folder

    "n_folds": 5,
    "random_state": 42,

    # === blending & feature flags ===
    "use_length_features": False,
    "use_weighted_logit_blend": True,
    "use_stacking_lr": True,
    "stack_use_engineered_features": True,

    # === numeric constants ===
    "epsilon": 1e-6,

    # === optimizer ===
    "weight_init": "equal",
    "weight_opt_max_iter": 800,
    "weight_opt_tolerance": 1e-8,

    # === stacking config ===
    "stack_C": 2.0,
    "stack_max_iter": 500,
    "stack_solver": "lbfgs",
}


In [5]:
# ---------------------------------
# LOGGING
# ---------------------------------
def setup_logging():
    logging.basicConfig(
        level=logging.INFO,
        format="[%(asctime)s] %(levelname)s - %(message)s",
        datefmt="%H:%M:%S",
        handlers=[logging.StreamHandler(sys.stdout)],
    )

In [6]:
setup_logging()

In [7]:
# ---------------------------------
# UTILS
# ---------------------------------
def softmax(logits: np.ndarray, axis: int = -1) -> np.ndarray:
    x = logits - logits.max(axis=axis, keepdims=True)
    e = np.exp(x)
    return e / (e.sum(axis=axis, keepdims=True) + 1e-12)

In [8]:
def probs_to_logits(p: np.ndarray, eps: float = 1e-9) -> np.ndarray:
    """
    Multi-class safe logit transform: returns class-wise log-probs (unnormalized).
    For blending we prefer to work in (natural) logit space per class via log(p).
    Here we use log(p) and renormalize later with softmax after weighting.

    p: [..., 3] (A,B,Tie) probs
    """
    p = np.clip(p, eps, 1 - eps)
    return np.log(p)

In [9]:
def logits_to_probs(logits: np.ndarray) -> np.ndarray:
    return softmax(logits, axis=-1)

In [10]:
def check_prob_frame(df: pd.DataFrame, name: str):
    required = ["id", "pA", "pB", "pTie"]
    for c in required:
        if c not in df.columns:
            raise ValueError(f"{name}: missing column '{c}'")
    vals = df[["pA", "pB", "pTie"]].values
    if np.isnan(vals).any() or np.isinf(vals).any():
        raise ValueError(f"{name}: found NaN/Inf probabilities.")
    row_sums = vals.sum(axis=1)
    bad = np.where((row_sums < 0.999 - 1e-5) | (row_sums > 1.001 + 1e-5))[0]
    if len(bad) > 0:
        logging.warning(f"{name}: {len(bad)} rows do not sum to ~1. Will renormalize softly.")
        # gentle renorm
        vals = np.clip(vals, 0, 1)
        vals = vals / (vals.sum(axis=1, keepdims=True) + 1e-12)
        df[["pA", "pB", "pTie"]] = vals
    return df

In [11]:
def safe_inner_merge(a: pd.DataFrame, b: pd.DataFrame, on: str, name_a: str, name_b: str) -> pd.DataFrame:
    before = len(a)
    out = a.merge(b, on=on, how="inner", suffixes=("", f"__{name_b}"))
    after = len(out)
    if after < before:
        logging.warning(f"Merge({name_a} x {name_b}) dropped {before-after} rows (inner join on '{on}').")
    return out

In [12]:
def label_from_winners(r: pd.Series) -> int:
    # 0 = A, 1 = B, 2 = Tie
    if r["winner_model_a"] == 1:
        return 0
    if r["winner_model_b"] == 1:
        return 1
    return 2

In [13]:
def entropy_rows(p: np.ndarray, eps: float = 1e-12) -> np.ndarray:
    p = np.clip(p, eps, 1.0)
    return -(p * np.log(p)).sum(axis=1)

In [14]:
def make_prompt_groups(df: pd.DataFrame) -> np.ndarray:
    # Stable hash for GroupKFold by prompt
    return df["prompt"].astype(str).apply(lambda x: hash(x) % (10**9)).values

In [15]:
def ab_swap(df_probs: pd.DataFrame) -> pd.DataFrame:
    swapped = df_probs.copy()
    swapped = swapped.rename(columns={"pA": "pB_temp", "pB": "pA_temp"})
    swapped["pA"] = swapped["pA_temp"]
    swapped["pB"] = swapped["pB_temp"]
    swapped.drop(columns=["pA_temp", "pB_temp"], inplace=True)
    # Tie unchanged
    return swapped

In [16]:
def ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)

In [17]:
# ---------------------------------
# LOADING DATA
# ---------------------------------
def load_official(train_csv: str, test_csv: str, use_len_feats: bool) -> Tuple[pd.DataFrame, pd.DataFrame]:
    train = pd.read_csv(train_csv)
    test  = pd.read_csv(test_csv)
    # labels 0/1/2
    train["label"] = train.apply(label_from_winners, axis=1)

    # lengths (optional)
    if use_len_feats:
        for df in (train, test):
            for c in ("response_a", "response_b", "prompt"):
                df[f"{c}_len"] = df[c].astype(str).str.len()
    return train, test

In [18]:
@dataclass
class ModelPreds:
    name: str
    train: Optional[pd.DataFrame]
    test:  Optional[pd.DataFrame]

In [19]:
'''
def try_load_model_preds(spec: Dict) -> ModelPreds:
    name = spec["name"]
    tr_path = spec.get("train_path", None)
    te_path = spec.get("test_path", None)

    tr_df = None
    te_df = None
    if tr_path and os.path.exists(tr_path):
        tr_df = pd.read_csv(tr_path)
        tr_df = check_prob_frame(tr_df, f"{name}::train")
    else:
        logging.warning(f"[{name}] OOF train file not found -> skipping train for this model.")

    if te_path and os.path.exists(te_path):
        te_df = pd.read_csv(te_path)
        te_df = check_prob_frame(te_df, f"{name}::test")
    else:
        logging.warning(f"[{name}] test file not found -> skipping test for this model.")

    return ModelPreds(name=name, train=tr_df, test=te_df)'''

def try_load_model_preds(spec: Dict) -> ModelPreds:
    name = spec["name"]
    tr_path = spec.get("train_path", None)
    te_path = spec.get("test_path", None)

    tr_df = None
    te_df = None

    if tr_path and os.path.exists(tr_path):
        tr_df = pd.read_csv(tr_path)
    else:
        logging.warning(f"[{name}] OOF train file not found -> skipping train for this model.")

    if te_path and os.path.exists(te_path):
        te_df = pd.read_csv(te_path)
    else:
        logging.warning(f"[{name}] test file not found -> skipping test for this model.")

    #Standardize column names so DeBERTa submissions work
    rename_map = {
        "winner_model_a": "pA",
        "winner_model_b": "pB",
        "winner_tie": "pTie"
    }
    if tr_df is not None:
        tr_df.rename(columns=rename_map, inplace=True)
    if te_df is not None:
        te_df.rename(columns=rename_map, inplace=True)

    # Run safety checks if applicable
    if tr_df is not None:
        tr_df = check_prob_frame(tr_df, f"{name}::train")
    if te_df is not None:
        te_df = check_prob_frame(te_df, f"{name}::test")

    return ModelPreds(name=name, train=tr_df, test=te_df)

In [20]:
# ---------------------------------
# FEATURE ENGINEERING
# ---------------------------------
def build_feature_matrices(
    train_df: pd.DataFrame,
    test_df: pd.DataFrame,
    preds_list: List[ModelPreds],
    use_engineered: bool = True,
    use_len_feats: bool = True,
    eps: float = 1e-9,
) -> Tuple[pd.DataFrame, pd.DataFrame, List[str]]:
    """
    Returns:
      X_train, X_test, feat_names
    Each contains concatenated per-model probability channels + engineered features.
    """
    base_cols_train = ["id", "label", "prompt"]
    if not set(base_cols_train).issubset(train_df.columns):
        raise ValueError("train_df must contain id, label, prompt")

    # Start with the ID and target; we’ll add features to side DataFrames and join by id
    Xtr = train_df[["id"]].copy()
    Xte = test_df[["id"]].copy()

    feat_names: List[str] = []

    # Concatenate all model probs with clear prefixes
    for mp in preds_list:
        if mp.train is None and mp.test is None:
            continue
        mname = mp.name
        if mp.train is not None:
            tmp = mp.train[["id", "pA", "pB", "pTie"]].copy()
            tmp = tmp.rename(columns={"pA": f"{mname}_pA", "pB": f"{mname}_pB", "pTie": f"{mname}_pTie"})
            Xtr = safe_inner_merge(Xtr, tmp, on="id", name_a="Xtr", name_b=mname)
        if mp.test is not None:
            tmp = mp.test[["id", "pA", "pB", "pTie"]].copy()
            tmp = tmp.rename(columns={"pA": f"{mname}_pA", "pB": f"{mname}_pB", "pTie": f"{mname}_pTie"})
            Xte = safe_inner_merge(Xte, tmp, on="id", name_a="Xte", name_b=mname)

    # Track the per-model prob features
    for mp in preds_list:
        mname = mp.name
        for ch in ["pA", "pB", "pTie"]:
            col = f"{mname}_{ch}"
            if col in Xtr.columns:
                feat_names.append(col)

    # Engineered features per model (margins, entropy, confidence)
    if use_engineered:
        for mp in preds_list:
            mname = mp.name
            pAcol, pBcol, pTcol = f"{mname}_pA", f"{mname}_pB", f"{mname}_pTie"
            if all(col in Xtr.columns for col in [pAcol, pBcol, pTcol]):
                for df, label in [(Xtr, "tr"), (Xte, "te")]:
                    # margins
                    df[f"{mname}_margin_AB"] = df[pAcol] - df[pBcol]
                    df[f"{mname}_margin_maxTie"] = df[pTcol] - np.maximum(df[pAcol], df[pBcol])
                    # max prob & entropy
                    probs = df[[pAcol, pBcol, pTcol]].values
                    df[f"{mname}_maxprob"] = probs.max(axis=1)
                    df[f"{mname}_entropy"] = entropy_rows(probs, eps=eps)
                feat_names += [f"{mname}_margin_AB", f"{mname}_margin_maxTie", f"{mname}_maxprob", f"{mname}_entropy"]

    # Length features (from raw train/test)
    if use_len_feats:
        for col in ["response_a_len", "response_b_len", "prompt_len"]:
            if col in train_df.columns and col in test_df.columns:
                Xtr[col] = train_df[col].values
                Xte[col] = test_df[col].values
                feat_names.append(col)
        # simple interactions
        if all(c in Xtr.columns for c in ["response_a_len", "response_b_len"]):
            Xtr["len_sum"] = Xtr["response_a_len"] + Xtr["response_b_len"]
            Xtr["len_diff"] = np.abs(Xtr["response_a_len"] - Xtr["response_b_len"])
            Xte["len_sum"] = Xte["response_a_len"] + Xte["response_b_len"]
            Xte["len_diff"] = np.abs(Xte["response_a_len"] - Xte["response_b_len"])
            feat_names += ["len_sum", "len_diff"]

    # Fill any residual NaNs (e.g., a model missing in test)
    Xtr = Xtr.fillna(0.0)
    Xte = Xte.fillna(0.0)

    # Final safety: ensure all feat_names exist in Xtr/Xte
    feat_names = [f for f in feat_names if f in Xtr.columns and f in Xte.columns]

    return Xtr, Xte, feat_names

In [21]:
# ---------------------------------
# EVALUATION / CV
# ---------------------------------
def cv_splits(train_df: pd.DataFrame, n_folds: int) -> List[Tuple[np.ndarray, np.ndarray]]:
    groups = make_prompt_groups(train_df)
    gkf = GroupKFold(n_splits=n_folds)
    splits = []
    for tr_idx, va_idx in gkf.split(train_df, groups=groups):
        splits.append((tr_idx, va_idx))
    return splits

In [22]:
# ---------------------------------
# BLEND STRATEGY 1: Constrained logit-space weighted average
# ---------------------------------
@dataclass
class WeightedLogitBlend:
    model_names: List[str]
    weights: Optional[np.ndarray] = None   # shape [n_models, 3] or [n_models] if shared across classes
    classwise: bool = False                # if True, learn separate weights per class

    def _init_weights(self, n_models: int, classwise: bool, init: str) -> np.ndarray:
        if classwise:
            if init == "equal":
                W = np.ones((n_models, 3), dtype=np.float64) / n_models
            elif init == "prioritized_first":
                W = np.zeros((n_models, 3), dtype=np.float64)
                W[0, :] = 1.0
            else:  # random
                R = np.random.RandomState(42).rand(n_models, 3)
                W = R / (R.sum(axis=0, keepdims=True) + 1e-12)
        else:
            if init == "equal":
                w = np.ones((n_models,), dtype=np.float64) / n_models
            elif init == "prioritized_first":
                w = np.zeros((n_models,), dtype=np.float64)
                w[0] = 1.0
            else:
                R = np.random.RandomState(42).rand(n_models)
                w = R / (R.sum() + 1e-12)
            W = w
        return W

    def _param_to_weights(self, theta: np.ndarray, n_models: int, classwise: bool) -> np.ndarray:
        """
        Map unconstrained theta -> nonnegative weights that sum to 1 (per class if classwise)
        Use softmax transform.
        """
        if classwise:
            theta_2d = theta.reshape(n_models, 3)
            W = np.exp(theta_2d)
            W = W / (W.sum(axis=0, keepdims=True) + 1e-12)
        else:
            w = np.exp(theta)
            W = w / (w.sum() + 1e-12)
        return W

    def fit_cv(
        self,
        X: pd.DataFrame,
        feat_names: List[str],
        y: np.ndarray,
        splits: List[Tuple[np.ndarray, np.ndarray]],
        model_names_in_feats: List[str],
        max_iter: int = 800,
        tol: float = 1e-8,
        eps: float = 1e-9,
        init: str = "equal",
        classwise: bool = False,
    ) -> Dict:
        """
        X: contains stacked per-model prob features with names like "{model}_pA/B/Tie"
        model_names_in_feats: order of models to stack; must exist in feat_names
        """
        # gather per-model probs into 3D array: [N, n_models, 3]
        N = len(X)
        ms = model_names_in_feats
        n_models = len(ms)
        if n_models == 0:
            raise ValueError("WeightedLogitBlend: no models to blend.")

        P = np.zeros((N, n_models, 3), dtype=np.float64)
        for i, m in enumerate(ms):
            P[:, i, 0] = X[f"{m}_pA"].values
            P[:, i, 1] = X[f"{m}_pB"].values
            P[:, i, 2] = X[f"{m}_pTie"].values

        # convert probs to logits (per class)
        L = np.log(np.clip(P, eps, 1 - eps))  # shape [N, M, 3]

        cv_losses = []
        oof = np.zeros((N, 3), dtype=np.float64)
        best_thetas = []

        # initial theta for optimization
        if classwise:
            W0 = self._init_weights(n_models, True, init)  # [M,3]
            theta0 = np.log(np.clip(W0, 1e-8, 1.0))  # inverse softmax-ish seed
            theta0 = theta0.ravel()
        else:
            W0 = self._init_weights(n_models, False, init)  # [M]
            theta0 = np.log(np.clip(W0, 1e-8, 1.0))  # seed

        def fold_objective(theta, L_tr, y_tr):
            W = self._param_to_weights(theta, n_models, classwise)
            # combine logits
            if classwise:
                # for each class c, weighted sum L[:,:,c] with W[:,c]
                logits_comb = np.zeros((L_tr.shape[0], 3), dtype=np.float64)
                for c in range(3):
                    logits_comb[:, c] = (L_tr[:, :, c] @ W[:, c])
            else:
                # single weight vector shared for all classes
                logits_comb = np.tensordot(L_tr, W, axes=([1],[0]))  # [N,3]
            probs = softmax(logits_comb, axis=1)
            # log loss
            y_onehot = np.eye(3)[y_tr]
            loss = -np.mean((y_onehot * np.log(np.clip(probs, 1e-12, 1.0))).sum(axis=1))
            return loss

        for k, (tr_idx, va_idx) in enumerate(splits):
            L_tr, L_va = L[tr_idx], L[va_idx]
            y_tr, y_va = y[tr_idx], y[va_idx]

            res = minimize(
                fold_objective,
                x0=theta0,
                args=(L_tr, y_tr),
                method="L-BFGS-B",
                options={"maxiter": max_iter, "ftol": tol, "gtol": tol}
            )
            theta_star = res.x
            W_star = self._param_to_weights(theta_star, n_models, classwise)

            # validate
            if classwise:
                logits_va = np.zeros((len(va_idx), 3), dtype=np.float64)
                for c in range(3):
                    logits_va[:, c] = (L_va[:, :, c] @ W_star[:, c])
            else:
                logits_va = np.tensordot(L_va, W_star, axes=([1], [0]))
            probs_va = softmax(logits_va, axis=1)
            oof[va_idx] = probs_va
            fold_loss = log_loss(y_va, probs_va, labels=[0,1,2])
            cv_losses.append(fold_loss)
            best_thetas.append(theta_star)

        mean_cv = float(np.mean(cv_losses))
        std_cv  = float(np.std(cv_losses))

        # store the average theta across folds (simple mean; could also refit on full)
        theta_bar = np.mean(np.vstack(best_thetas), axis=0)
        self.weights = self._param_to_weights(theta_bar, n_models, classwise)
        self.classwise = classwise

        return {
            "cv_losses": cv_losses,
            "cv_mean": mean_cv,
            "cv_std":  std_cv,
            "oof_probs": oof,
            "weights_matrix": self.weights.tolist() if classwise else self.weights.tolist(),
            "classwise": classwise,
            "model_order": ms,
        }

    def predict(self, X: pd.DataFrame, model_names_in_feats: List[str], eps: float = 1e-9) -> np.ndarray:
        assert self.weights is not None, "Call fit_cv first."
        ms = model_names_in_feats
        n_models = len(ms)
        N = len(X)
        P = np.zeros((N, n_models, 3), dtype=np.float64)
        for i, m in enumerate(ms):
            P[:, i, 0] = X[f"{m}_pA"].values
            P[:, i, 1] = X[f"{m}_pB"].values
            P[:, i, 2] = X[f"{m}_pTie"].values
        L = np.log(np.clip(P, eps, 1 - eps))

        if self.classwise:
            logits = np.zeros((N, 3), dtype=np.float64)
            for c in range(3):
                logits[:, c] = (L[:, :, c] @ self.weights[:, c])
        else:
            logits = np.tensordot(L, self.weights, axes=([1], [0]))
        return softmax(logits, axis=1)

In [23]:
# ---------------------------------
# BLEND STRATEGY 2: Stacking (multinomial logistic regression)
# ---------------------------------
@dataclass
class StackingLR:
    feat_names: List[str]
    C: float = 2.0
    max_iter: int = 2000
    solver: str = "lbfgs"
    scaler: Optional[StandardScaler] = None
    clf: Optional[LogisticRegression] = None

    def fit_cv(self, X: pd.DataFrame, y: np.ndarray, splits: List[Tuple[np.ndarray, np.ndarray]]) -> Dict:
        if len(self.feat_names) == 0:
            raise ValueError("StackingLR: No features provided.")

        oof = np.zeros((len(X), 3), dtype=np.float64)
        cv_losses = []

        for k, (tr_idx, va_idx) in enumerate(splits):
            Xtr = X.iloc[tr_idx][self.feat_names].values
            Xva = X.iloc[va_idx][self.feat_names].values
            ytr = y[tr_idx]
            yva = y[va_idx]

            pipe = Pipeline([
                ("scaler", StandardScaler(with_mean=True, with_std=True)),
                ("clf", LogisticRegression(
                    C=self.C, max_iter=self.max_iter, solver=self.solver,
                    multi_class="multinomial"
                )),
            ])
            pipe.fit(Xtr, ytr)
            pva = pipe.predict_proba(Xva)
            oof[va_idx] = pva
            loss = log_loss(yva, pva, labels=[0,1,2])
            cv_losses.append(loss)

        mean_cv = float(np.mean(cv_losses))
        std_cv  = float(np.std(cv_losses))

        # Refit on full train
        self.clf = LogisticRegression(
            C=self.C, max_iter=self.max_iter, solver=self.solver, multi_class="multinomial"
        )
        self.scaler = StandardScaler(with_mean=True, with_std=True)
        Xfull = X[self.feat_names].values
        self.scaler.fit(Xfull)
        Xs = self.scaler.transform(Xfull)
        self.clf.fit(Xs, y)

        return {
            "cv_losses": cv_losses,
            "cv_mean": mean_cv,
            "cv_std":  std_cv,
            "oof_probs": oof,
        }

    def predict(self, X: pd.DataFrame) -> np.ndarray:
        assert self.clf is not None and self.scaler is not None, "Call fit_cv first."
        Xs = self.scaler.transform(X[self.feat_names].values)
        return self.clf.predict_proba(Xs)

In [24]:
# ---------------------------------
# SYMMETRY & CALIBRATION CHECKS
# ---------------------------------
def symmetry_unit_test(p_df: pd.DataFrame, eps: float = 1e-6):
    """
    If we swap A<->B probs at inputs for every model, the final predictions
    should also swap A<->B (tie unchanged). This function checks that behavior
    **for simple weighted averaging sanity**. For stacking, we test by swapping
    the per-model features accordingly and verifying predictions swap.
    """
    # This is provided as a placeholder for targeted tests if desired.
    # In practice, after producing final test predictions, we will duplicate
    # the pipeline, swap A/B in inputs, and verify output swap holds “close enough”.
    pass

In [25]:
def tie_rate_report(oof_probs: np.ndarray, y_true: np.ndarray) -> Dict:
    pred_tie = (oof_probs.argmax(axis=1) == 2).mean()
    true_tie = (y_true == 2).mean()
    return {"pred_tie_rate": float(pred_tie), "true_tie_rate": float(true_tie)}

In [ ]:
"""# ---------------------------------
# MAIN PIPELINE
# ---------------------------------
def main():
    parser = argparse.ArgumentParser(description="Combiner / Ensembler")
    parser.add_argument("--train_csv", type=str, default=CONFIG["train_csv"])
    parser.add_argument("--test_csv",  type=str, default=CONFIG["test_csv"])
    parser.add_argument("--runs_dir",  type=str, default=CONFIG["runs_dir"])
    parser.add_argument("--tag",       type=str, default=None, help="Optional run tag; else timestamped.")
    parser.add_argument("--classwise_weights", action="store_true", help="Learn classwise weights for logit blend.")
    args = parser.parse_args() if "__file__" in globals() else argparse.Namespace(
        train_csv=CONFIG["train_csv"],
        test_csv=CONFIG["test_csv"],
        runs_dir=CONFIG["runs_dir"],
        tag=None,
        classwise_weights=False,
    )

    np.random.seed(CONFIG["random_state"])

    # Output dirs
    run_tag = args.tag or time.strftime("%Y%m%d_%H%M%S")
    run_dir = os.path.join(args.runs_dir, run_tag)
    ensure_dir(run_dir)
    logging.info(f"Run dir: {run_dir}")

    # Load official
    logging.info("Loading official train/test...")
    train_df, test_df = load_official(args.train_csv, args.test_csv, CONFIG["use_length_features"])
    y = train_df["label"].values

    # Load model predictions
    logging.info("Loading model artifacts...")
    preds_list: List[ModelPreds] = []
    for spec in CONFIG["model_specs"]:
        mp = try_load_model_preds(spec)
        # Keep only if at least one side (train or test) exists
        if (mp.train is not None) or (mp.test is not None):
            preds_list.append(mp)
        else:
            logging.warning(f"Skipping model '{spec['name']}' (no files found).")

    if len(preds_list) == 0:
        raise RuntimeError("No model predictions found. Add at least one to CONFIG['model_specs'].")

    # Build features
    logging.info("Building meta features...")
    Xtr, Xte, feat_names = build_feature_matrices(
        train_df=train_df, test_df=test_df,
        preds_list=preds_list,
        use_engineered=CONFIG["stack_use_engineered_features"],
        use_len_feats=CONFIG["use_length_features"],
        eps=CONFIG["epsilon"],
    )

    # Which models are actually available in features?
    model_names_available = []
    for mp in preds_list:
        mname = mp.name
        if all(f"{mname}_{ch}" in feat_names for ch in ("pA","pB","pTie")):
            model_names_available.append(mname)
    logging.info(f"Models available for blending: {model_names_available}")
    if len(model_names_available) == 0:
        raise RuntimeError("No per-model probability features present. Check file paths/columns.")

    # Build CV splits
    logging.info(f"Creating GroupKFold(n={CONFIG['n_folds']}) splits by prompt...")
    splits = cv_splits(train_df, CONFIG["n_folds"])

    results_summary = {}
    oof_candidates = {}

    # Strategy 1: Weighted Logit Blend
    if CONFIG["use_weighted_logit_blend"]:
        logging.info("==> Strategy: Weighted Logit Blend (logit-space)")
        wblend = WeightedLogitBlend(model_names=model_names_available)
        wb_info = wblend.fit_cv(
            X=Xtr, feat_names=feat_names, y=y, splits=splits,
            model_names_in_feats=model_names_available,
            max_iter=CONFIG["weight_opt_max_iter"], tol=CONFIG["weight_opt_tolerance"],
            init=CONFIG["weight_init"], classwise=args.classwise_weights,
        )
        results_summary["weighted_logit_blend"] = {
            "cv_mean": wb_info["cv_mean"],
            "cv_std": wb_info["cv_std"],
            "cv_losses": wb_info["cv_losses"],
            "weights": wb_info["weights_matrix"],
            "classwise": wb_info["classwise"],
            "model_order": wb_info["model_order"],
        }
        oof_candidates["weighted_logit_blend"] = wb_info["oof_probs"]

    # Strategy 2: Stacking LR
    if CONFIG["use_stacking_lr"]:
        logging.info("==> Strategy: Stacking (Multinomial Logistic Regression)")
        # Stacking uses ALL feat_names (probs + engineered + lengths) by default
        stack = StackingLR(feat_names=feat_names, C=CONFIG["stack_C"],
                           max_iter=CONFIG["stack_max_iter"], solver=CONFIG["stack_solver"])
        st_info = stack.fit_cv(X=Xtr, y=y, splits=splits)
        results_summary["stacking_lr"] = {
            "cv_mean": st_info["cv_mean"],
            "cv_std": st_info["cv_std"],
            "cv_losses": st_info["cv_losses"],
        }
        oof_candidates["stacking_lr"] = st_info["oof_probs"]

    # Pick best strategy by mean CV
    if len(results_summary) == 0:
        raise RuntimeError("No strategies enabled. Enable at least one in CONFIG.")

    best_name = min(results_summary.keys(), key=lambda k: results_summary[k]["cv_mean"])
    best_cv = results_summary[best_name]["cv_mean"]
    logging.info(f"Best strategy: {best_name} (mean CV log-loss={best_cv:.6f})")

    # Produce final predictions on test using the chosen strategy
    if best_name == "weighted_logit_blend":
        final_model = WeightedLogitBlend(model_names=model_names_available)
        final_model.weights = np.array(results_summary["weighted_logit_blend"]["weights"])
        final_model.classwise = results_summary["weighted_logit_blend"]["classwise"]
        test_probs = final_model.predict(Xte, model_names_in_feats=model_names_available)
        oof_final = oof_candidates["weighted_logit_blend"]
    else:
        # already refit on full train inside StackingLR
        test_probs = stack.predict(Xte)
        oof_final = oof_candidates["stacking_lr"]

    # Diagnostics
    tie_stats = tie_rate_report(oof_final, y)
    logging.info(f"Tie rate (true): {tie_stats['true_tie_rate']:.4f} | (pred OOF): {tie_stats['pred_tie_rate']:.4f}")

    # OOF file
    oof_df = pd.DataFrame({
        "id": Xtr["id"].values,
        "pA": oof_final[:, 0],
        "pB": oof_final[:, 1],
        "pTie": oof_final[:, 2],
        "label": y,
    })
    oof_path = os.path.join(run_dir, "oof_combined.csv")
    oof_df.to_csv(oof_path, index=False)
    logging.info(f"Saved OOF combined -> {oof_path}")

    # CV report
    cv_report_path = os.path.join(run_dir, "cv_report.json")
    with open(cv_report_path, "w") as f:
        json.dump({
            "strategies": results_summary,
            "chosen": best_name,
            "tie_stats": tie_stats,
            "n_models_used": len(model_names_available),
            "models_used": model_names_available,
            "n_folds": CONFIG["n_folds"],
        }, f, indent=2)
    logging.info(f"Saved CV report -> {cv_report_path}")

    # Blend config
    blend_cfg_path = os.path.join(run_dir, "blend_config.json")
    chosen_cfg = {
        "chosen_strategy": best_name,
        "feat_names": feat_names,
        "model_names_available": model_names_available,
        "config": CONFIG,
    }
    if best_name == "weighted_logit_blend":
        chosen_cfg["weights"] = results_summary["weighted_logit_blend"]["weights"]
        chosen_cfg["classwise"] = results_summary["weighted_logit_blend"]["classwise"]
    with open(blend_cfg_path, "w") as f:
        json.dump(chosen_cfg, f, indent=2)
    logging.info(f"Saved blend config -> {blend_cfg_path}")

    # Final Test Submission
    sub = pd.DataFrame({
        "id": Xte["id"].values,
        "winner_model_a": test_probs[:, 0],
        "winner_model_b": test_probs[:, 1],
        "winner_tie":     test_probs[:, 2],
    })
    # gentle renorm guard
    sums = sub[["winner_model_a","winner_model_b","winner_tie"]].sum(axis=1).values
    sub[["winner_model_a","winner_model_b","winner_tie"]] = sub[["winner_model_a","winner_model_b","winner_tie"]].values / (sums[:, None] + 1e-12)
    sub_path = os.path.join(run_dir, "submission.csv")
    sub.to_csv(sub_path, index=False)
    logging.info(f"Saved submission -> {sub_path}")

    # Quick symmetry sanity check (best-effort)
    try:
        # Create an A/B-swapped copy of per-model probs in test features,
        # recompute final predictions, and verify swap at the output.
        Xte_sw = Xte.copy()
        # Swap all *model*_pA <-> *model*_pB features
        for m in model_names_available:
            a, b = f"{m}_pA", f"{m}_pB"
            if a in Xte_sw.columns and b in Xte_sw.columns:
                tmp = Xte_sw[a].copy()
                Xte_sw[a] = Xte_sw[b]
                Xte_sw[b] = tmp
        # The engineered features involving margins were built on original probs,
        # so for a rigorous test you'd rebuild engineered features for swapped inputs.
        # For a quick check, if using stacking with engineered feats, skip deep check.
        if best_name == "weighted_logit_blend":
            swapped_probs = final_model.predict(Xte_sw, model_names_in_feats=model_names_available)
            # Compare to the original: new pA should ~ old pB, new pB ~ old pA, tie ~ tie
            deltaA = np.mean(np.abs(swapped_probs[:,0] - test_probs[:,1]))
            deltaB = np.mean(np.abs(swapped_probs[:,1] - test_probs[:,0]))
            deltaT = np.mean(np.abs(swapped_probs[:,2] - test_probs[:,2]))
            logging.info(f"[Symmetry check] mean |pA_sw - pB|={deltaA:.6f}, |pB_sw - pA|={deltaB:.6f}, |pTie_sw - pTie|={deltaT:.6f}")
        else:
            logging.info("[Symmetry check] Skipped strict swap test for stacking (engineered feats not rebuilt).")
    except Exception as e:
        logging.warning(f"Symmetry check failed/omitted: {e}")

    logging.info("Done.")"""

In [ ]:
# ---------------------------------
# MAIN PIPELINE
# ---------------------------------
def main():
    parser = argparse.ArgumentParser(description="Combiner / Ensembler")
    parser.add_argument("--train_csv", type=str, default=CONFIG["train_csv"])
    parser.add_argument("--test_csv",  type=str, default=CONFIG["test_csv"])
    parser.add_argument("--runs_dir",  type=str, default=CONFIG["runs_dir"])
    parser.add_argument("--tag",       type=str, default=None, help="Optional run tag; else timestamped.")
    parser.add_argument("--classwise_weights", action="store_true", help="Learn classwise weights for logit blend.")
    args = parser.parse_args() if "__file__" in globals() else argparse.Namespace(
        train_csv=CONFIG["train_csv"],
        test_csv=CONFIG["test_csv"],
        runs_dir=CONFIG["runs_dir"],
        tag=None,
        classwise_weights=False,
    )

    np.random.seed(CONFIG["random_state"])

    # Output dirs
    run_tag = args.tag or time.strftime("%Y%m%d_%H%M%S")
    run_dir = os.path.join(args.runs_dir, run_tag)
    ensure_dir(run_dir)
    logging.info(f"Run dir: {run_dir}")

    # Load official
    logging.info("Loading official train/test...")
    train_df, test_df = load_official(args.train_csv, args.test_csv, CONFIG["use_length_features"])
    y = train_df["label"].values

    # Load model predictions
    logging.info("Loading model artifacts...")
    preds_list: List[ModelPreds] = []
    for spec in CONFIG["model_specs"]:
        mp = try_load_model_preds(spec)
        # Keep only if at least one side (train or test) exists
        if (mp.train is not None) or (mp.test is not None):
            preds_list.append(mp)
        else:
            logging.warning(f"Skipping model '{spec['name']}' (no files found).")

    if len(preds_list) == 0:
        raise RuntimeError("No model predictions found. Add at least one to CONFIG['model_specs'].")

    # Build features
    logging.info("Building meta features...")
    Xtr, Xte, feat_names = build_feature_matrices(
        train_df=train_df, test_df=test_df,
        preds_list=preds_list,
        use_engineered=CONFIG["stack_use_engineered_features"],
        use_len_feats=CONFIG["use_length_features"],
        eps=CONFIG["epsilon"],
    )

    # Which models are actually available in features?
    model_names_available = []
    for mp in preds_list:
        mname = mp.name
        if all(f"{mname}_{ch}" in feat_names for ch in ("pA","pB","pTie")):
            model_names_available.append(mname)
    logging.info(f"Models available for blending: {model_names_available}")
    if len(model_names_available) == 0:
        raise RuntimeError("No per-model probability features present. Check file paths/columns.")

    # Build CV splits
    logging.info(f"Creating GroupKFold(n={CONFIG['n_folds']}) splits by prompt...")
    splits = cv_splits(train_df, CONFIG["n_folds"])

    results_summary = {}
    oof_candidates = {}

    # Strategy 1: Weighted Logit Blend
    if CONFIG["use_weighted_logit_blend"]:
        logging.info("==> Strategy: Weighted Logit Blend (logit-space)")
        wblend = WeightedLogitBlend(model_names=model_names_available)
        wb_info = wblend.fit_cv(
            X=Xtr, feat_names=feat_names, y=y, splits=splits,
            model_names_in_feats=model_names_available,
            max_iter=CONFIG["weight_opt_max_iter"], tol=CONFIG["weight_opt_tolerance"],
            init=CONFIG["weight_init"], classwise=args.classwise_weights,
        )
        results_summary["weighted_logit_blend"] = {
            "cv_mean": wb_info["cv_mean"],
            "cv_std": wb_info["cv_std"],
            "cv_losses": wb_info["cv_losses"],
            "weights": wb_info["weights_matrix"],
            "classwise": wb_info["classwise"],
            "model_order": wb_info["model_order"],
        }
        oof_candidates["weighted_logit_blend"] = wb_info["oof_probs"]

    # Strategy 2: Stacking LR
    if CONFIG["use_stacking_lr"]:
        logging.info("==> Strategy: Stacking (Multinomial Logistic Regression)")
        stack = StackingLR(feat_names=feat_names, C=CONFIG["stack_C"],
                           max_iter=CONFIG["stack_max_iter"], solver=CONFIG["stack_solver"])
        st_info = stack.fit_cv(X=Xtr, y=y, splits=splits)
        results_summary["stacking_lr"] = {
            "cv_mean": st_info["cv_mean"],
            "cv_std": st_info["cv_std"],
            "cv_losses": st_info["cv_losses"],
        }
        oof_candidates["stacking_lr"] = st_info["oof_probs"]

    # Pick best strategy by mean CV
    if len(results_summary) == 0:
        raise RuntimeError("No strategies enabled. Enable at least one in CONFIG.")

    best_name = min(results_summary.keys(), key=lambda k: results_summary[k]["cv_mean"])
    best_cv = results_summary[best_name]["cv_mean"]
    logging.info(f"Best strategy: {best_name} (mean CV log-loss={best_cv:.6f})")

    # Produce final predictions on test using the chosen strategy
    if best_name == "weighted_logit_blend":
        final_model = WeightedLogitBlend(model_names=model_names_available)
        final_model.weights = np.array(results_summary["weighted_logit_blend"]["weights"])
        final_model.classwise = results_summary["weighted_logit_blend"]["classwise"]
        test_probs = final_model.predict(Xte, model_names_in_feats=model_names_available)
        oof_final = oof_candidates["weighted_logit_blend"]
    else:
        test_probs = stack.predict(Xte)
        oof_final = oof_candidates["stacking_lr"]

    # Diagnostics
    tie_stats = tie_rate_report(oof_final, y)
    logging.info(f"Tie rate (true): {tie_stats['true_tie_rate']:.4f} | (pred OOF): {tie_stats['pred_tie_rate']:.4f}")

    # OOF file
    oof_df = pd.DataFrame({
        "id": Xtr["id"].values,
        "pA": oof_final[:, 0],
        "pB": oof_final[:, 1],
        "pTie": oof_final[:, 2],
        "label": y,
    })
    oof_path = os.path.join(run_dir, "oof_combined.csv")
    oof_df.to_csv(oof_path, index=False)
    logging.info(f"Saved OOF combined -> {oof_path}")

    # CV report
    cv_report_path = os.path.join(run_dir, "cv_report.json")
    with open(cv_report_path, "w") as f:
        json.dump({
            "strategies": results_summary,
            "chosen": best_name,
            "tie_stats": tie_stats,
            "n_models_used": len(model_names_available),
            "models_used": model_names_available,
            "n_folds": CONFIG["n_folds"],
        }, f, indent=2)
    logging.info(f"Saved CV report -> {cv_report_path}")

    # Blend config
    blend_cfg_path = os.path.join(run_dir, "blend_config.json")
    chosen_cfg = {
        "chosen_strategy": best_name,
        "feat_names": feat_names,
        "model_names_available": model_names_available,
        "config": CONFIG,
    }
    if best_name == "weighted_logit_blend":
        chosen_cfg["weights"] = results_summary["weighted_logit_blend"]["weights"]
        chosen_cfg["classwise"] = results_summary["weighted_logit_blend"]["classwise"]
    with open(blend_cfg_path, "w") as f:
        json.dump(chosen_cfg, f, indent=2)
    logging.info(f"Saved blend config -> {blend_cfg_path}")

    # Final Test Submission
    sub = pd.DataFrame({
        "id": Xte["id"].values,
        "winner_model_a": test_probs[:, 0],
        "winner_model_b": test_probs[:, 1],
        "winner_tie":     test_probs[:, 2],
    })
    sums = sub[["winner_model_a","winner_model_b","winner_tie"]].sum(axis=1).values
    sub[["winner_model_a","winner_model_b","winner_tie"]] = sub[["winner_model_a","winner_model_b","winner_tie"]].values / (sums[:, None] + 1e-12)
    sub_path = os.path.join(run_dir, "submission.csv")
    sub.to_csv(sub_path, index=False)
    logging.info(f"Saved submission -> {sub_path}")

    # Quick symmetry sanity check (best-effort)
    try:
        Xte_sw = Xte.copy()
        for m in model_names_available:
            a, b = f"{m}_pA", f"{m}_pB"
            if a in Xte_sw.columns and b in Xte_sw.columns:
                tmp = Xte_sw[a].copy()
                Xte_sw[a] = Xte_sw[b]
                Xte_sw[b] = tmp
        if best_name == "weighted_logit_blend":
            swapped_probs = final_model.predict(Xte_sw, model_names_in_feats=model_names_available)
            deltaA = np.mean(np.abs(swapped_probs[:,0] - test_probs[:,1]))
            deltaB = np.mean(np.abs(swapped_probs[:,1] - test_probs[:,0]))
            deltaT = np.mean(np.abs(swapped_probs[:,2] - test_probs[:,2]))
            logging.info(f"[Symmetry check] mean |pA_sw - pB|={deltaA:.6f}, |pB_sw - pA|={deltaB:.6f}, |pTie_sw - pTie|={deltaT:.6f}")
        else:
            logging.info("[Symmetry check] Skipped strict swap test for stacking (engineered feats not rebuilt).")
    except Exception as e:
        logging.warning(f"Symmetry check failed/omitted: {e}")

    # ---------------------------------
    # SUMMARY PRINTOUT (final weights + CV score)
    # ---------------------------------
    try:
        print("\n=== FINAL ENSEMBLE SUMMARY ===")
        if "weighted_logit_blend" in results_summary:
            weights = results_summary["weighted_logit_blend"]["weights"]
            model_order = results_summary["weighted_logit_blend"]["model_order"]
            mean_cv = results_summary["weighted_logit_blend"]["cv_mean"]
            std_cv  = results_summary["weighted_logit_blend"]["cv_std"]

            for name, w in zip(model_order, weights):
                print(f"{name}: {w:.2f}")
            print(f"Mean CV log-loss: {mean_cv:.3f} ± {std_cv:.3f}")

            summary_path = os.path.join(run_dir, "ensemble_summary.txt")
            with open(summary_path, "w") as f:
                f.write("=== FINAL ENSEMBLE SUMMARY ===\n")
                for name, w in zip(model_order, weights):
                    f.write(f"{name}: {w:.2f}\n")
                f.write(f"Mean CV log-loss: {mean_cv:.3f} ± {std_cv:.3f}\n")
            print(f"Saved summary -> {summary_path}")
        else:
            print("No weighted_logit_blend results available.")
    except Exception as e:
        logging.warning(f"Could not print ensemble summary: {e}")

    logging.info("Done.")


In [27]:
main()

[02:18:59] INFO - Run dir: ./runs/20251113_021859
[02:18:59] INFO - Loading official train/test...
[02:19:01] INFO - Loading model artifacts...
[02:19:01] INFO - Building meta features...
[02:19:01] INFO - Models available for blending: ['teacher2_reward', 'deberta_head_only']
[02:19:01] INFO - Creating GroupKFold(n=5) splits by prompt...
[02:19:01] INFO - ==> Strategy: Weighted Logit Blend (logit-space)
[02:19:02] INFO - ==> Strategy: Stacking (Multinomial Logistic Regression)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/Library/Frameworks/Pytho

[02:19:04] INFO - Best strategy: stacking_lr (mean CV log-loss=1.005790)
[02:19:04] INFO - Tie rate (true): 0.3090 | (pred OOF): 0.2435
[02:19:04] INFO - Saved OOF combined -> ./runs/20251113_021859/oof_combined.csv
[02:19:04] INFO - Saved CV report -> ./runs/20251113_021859/cv_report.json
[02:19:04] INFO - Saved blend config -> ./runs/20251113_021859/blend_config.json
[02:19:04] INFO - Saved submission -> ./runs/20251113_021859/submission.csv
[02:19:04] INFO - [Symmetry check] Skipped strict swap test for stacking (engineered feats not rebuilt).
[02:19:04] INFO - Done.


In [29]:
import pandas as pd

# Load the generated submission file
df = pd.read_csv("runs/20251113_021859/submission.csv")

# Print its shape and a few sample rows
print("Submission shape:", df.shape)
print(df.head())

Submission shape: (3, 4)
        id  winner_model_a  winner_model_b  winner_tie
0   136060        0.329364        0.276053    0.394583
1   211333        0.292047        0.410935    0.297018
2  1233961        0.204762        0.454993    0.340245


In [30]:
import pandas as pd

# check all key files used in CONFIG
files = {
    "train": "./data/train.csv",
    "test": "./data/test.csv",
    "teacher_train": "./teacher_logits_train_improved.csv",
    "teacher_test": "./teacher_logits_test_improved.csv",
    "deberta_train": "./deberta_oof_train.csv",
    "deberta_test": "./submission_deberta_small_calibrated.csv",
}

for name, path in files.items():
    try:
        df = pd.read_csv(path)
        print(f"{name:20s}: {df.shape}")
    except Exception as e:
        print(f"{name:20s}: ERROR → {e}")

train               : (57477, 9)
test                : (3, 4)
teacher_train       : (57477, 5)
teacher_test        : (3, 4)
deberta_train       : (57477, 5)
deberta_test        : (3, 4)
